In [1]:
import  os
import cv2
import numpy as np

In [2]:
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
import os

gauth = GoogleAuth()
gauth.LocalWebserverAuth()       
drive = GoogleDrive(gauth)

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
#import keras
import tensorflow as tf
from tensorflow.keras.layers import Layer,InputLayer,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D,MaxPool2D,Dropout,Reshape,Add,Conv2DTranspose,Concatenate,Lambda
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.patches as patches
from tensorflow.python.keras.engine.input_layer import Input
import tensorflow_probability as tfp


def normalSampling(var):
    normal_dist =  tfp.distributions.Normal(loc=var[0], scale=var[1])
    sampled = tf.squeeze(normal_dist.sample(1), axis=0)
    return sampled

def clipSteer(val):
    return  tf.clip_by_value(val,-5,5)
def clipSpeed(val):
    return  tf.clip_by_value(val,0,100)

def value_function():
    model = Sequential()
    model.add(InputLayer(input_shape=(64,64,3)))
    #1*1 layer
    model.add(Conv2D(3,1,activation="elu"))
    
    #model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(32,3,activation="elu"))
    #model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(32,3,activation="elu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    
    
    #model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(64,3,activation="elu"))
    #model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(64,3,activation="elu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    #model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(128,3,activation="elu"))
    #model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(128,3,activation="elu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    
    #model.add(Dense(1024,activation = "elu"))
    model.add(Dense(512,activation = "elu"))
    model.add(Dense(64,activation = "elu"))
    model.add(Dense(16,activation = "elu"))
    
    #output steer and speed
    model.add(Dense(1,activation = "elu"))
    
    
    
    return model


def policy():
    X_input=Input(shape = (64,64,3))
    
    x = Conv2D(3,1,activation="elu")(X_input)
    
    x = Conv2D(32,3,activation="elu")(x)
    x = Conv2D(32,3,activation="elu")(x)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Dropout(0.5)(x)
    
    x = Conv2D(64,3,activation="elu")(x)
    x = Conv2D(64,3,activation="elu")(x)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Dropout(0.5)(x)
    
    x = Conv2D(128,3,activation="elu")(x)
    x = Conv2D(128,3,activation="elu")(x)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Dropout(0.5)(x)
    
    x = Flatten()(x)
    xSplit = Dense(512,activation = "elu")(x)
    #xSplit = Dense(64,activation = "elu")(x)
    
    #steer
    xSteer = Dense(64,activation = "elu")(xSplit)
    xSteer = Dense(16,activation = "elu")(xSteer)
    muSteer = Dense(1,activation=None,name = "muSteer")(xSteer)
    sigmaSteer =  Dense(1,activation="softplus")(xSteer)
    sigmaSteer = Add(name = "sigmaSteer")([sigmaSteer,tf.constant([1e-5])])
    #steerDist =  tfp.distributions.Normal(muSteer, sigmaSteer,name = "speedDist")
    steer = Lambda(normalSampling,1)((muSteer,sigmaSteer))
    steer = Lambda(clipSteer,1,name = "steer")(steer)
    #speed
    xSpeed = Add()([steer,xSplit])
    xSpeed = Dense(64,activation = "elu")(xSplit)
    xSpeed = Dense(16,activation = "elu")(xSpeed)
    muSpeed = Dense(1,activation=None,name = "muSpeed")(xSpeed)
    sigmaSpeed =  Dense(1,activation="softplus")(xSpeed)
    sigmaSpeed = Add(name = "sigmaSpeed")([sigmaSpeed,tf.constant([1e-5])])
    #speedDist =  tfp.distributions.Normal(muSpeed, sigmaSpeed,name = "steerDist")
    speed = Lambda(normalSampling,1)((muSpeed,sigmaSpeed))
    speed = Lambda(clipSpeed,1,name="speed")(speed)
    
    model=Model(inputs=X_input,outputs =[speed,steer,muSpeed,sigmaSpeed,muSteer,sigmaSteer])
    return model 

def policy_loss(self,mu,sigma,td):
      def keras_loss(y_true,y_pred):
          normal_dist = tfp.distributions.Normal(mu,sigma)
          return -1* tf.math.log(normal_dist.prob(y_pred)+1e-5)*td
      return keras_loss

In [4]:
value_model = value_function()
value_model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),loss=tf.keras.losses.MeanSquaredError())
policy_model = policy()

In [9]:
iteration = 0
prev_state = None
gamma = 0.99
speed,steer,muSpeed,sigmaSpeed,muSteer,sigmaSteer = [None for x in range(6)]


while(True):
  if (os.path.isfile(f"/content/drive/MyDrive/RL4NXP/1_{iteration}_2.jpeg")):
    img = cv2.imread(f"/content/drive/MyDrive/RL4NXP/1_{iteration}_2.jpeg")
    speed,steer,muSpeed,sigmaSpeed,muSteer,sigmaSteer = policy_model.predict(img)
    
    file1 = drive.CreateFile({'title': f"1_{iteration}_2.txt" , 'parents':[{'id':'1U1tIilfX2rzKiQidcmEQEgaATi-Y2trt'}]})
    file1.SetContentString(f"{float(speed)} {float(steer)} {float(muSpeed)} {float(sigmaSpeed)} {float(muSteer)} {float(sigmaSteer)}")
    file1.Upload()

    prev_state = img 
    iteration += 1
  elif (os.path.isfile(f"/content/drive/MyDrive/RL4NXP/0_{iteration}_2.jpeg")):
    img = cv2.imread(f"/content/drive/MyDrive/RL4NXP/0_{iteration}_2.jpeg")
    prev_V = value_model.predict(prev_state)
    current_V = value_model.predict(img)
    
    file1 = open(f"/content/drive/MyDrive/RL4NXP/0_{iteration}_2R.txt","r")
    reward=float (file1.readline())


    target = reward + gamma*current_V
    td_error = target - prev_V
    value_model.fit(x= prev_state, y = target)

    losses={'speed':policy_loss(muSpeed,sigmaSpeed,td_error),'steer':policy_loss(muSteer,sigmaSteer,td_error),"muSpeed" :policy_loss(muSteer,sigmaSteer,td_error),"sigmaSpeed" :policy_loss(muSteer,sigmaSteer,td_error),"muSteer":policy_loss(muSteer,sigmaSteer,td_error),"sigmaSteer":policy_loss(muSteer,sigmaSteer,td_error)}
    lossWeights={'speed':1,'steer':1,"muSpeed":0,"sigmaSpeed":0,"muSteer":0,"sigmaSteer":0}
    
    policy_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss=losses,loss_weights=lossWeights)
    policy_model.fit(prev_state,[np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1)])
    speed,steer,muSpeed,sigmaSpeed,muSteer,sigmaSteer = policy_model.predict(img)
    
    file1 = drive.CreateFile({'title': f"0_{iteration}_2.txt" , 'parents':[{'id':'1U1tIilfX2rzKiQidcmEQEgaATi-Y2trt'}]})
    file1.SetContentString(f"{float(speed)} {float(steer)} {float(muSpeed)} {float(sigmaSpeed)} {float(muSteer)} {float(sigmaSteer)}")
    file1.Upload()

    prev_state = img
    iteration += 1


  elif (os.path.isfile(f"/content/drive/MyDrive/RL4NXP/0_{iteration}_0.jpeg")):

    passedImage = cv2.imread(f"/content/drive/MyDrive/RL4NXP/0_{iteration}_0.jpeg")

    prev_V = value_model.predict(prev_state)
    current_V = value_model.predict(passedImage)

    reward = -1500
    target = reward + gamma*current_V
    td_error = target - prev_V
    value_model.fit(x= prev_state, y = target)

    losses={'speed':policy_loss(muSpeed,sigmaSpeed,td_error),'steer':policy_loss(muSteer,sigmaSteer,td_error),"muSpeed" :policy_loss(muSteer,sigmaSteer,td_error),"sigmaSpeed" :policy_loss(muSteer,sigmaSteer,td_error),"muSteer":policy_loss(muSteer,sigmaSteer,td_error),"sigmaSteer":policy_loss(muSteer,sigmaSteer,td_error)}
    lossWeights={'speed':1,'steer':1,"muSpeed":0,"sigmaSpeed":0,"muSteer":0,"sigmaSteer":0}
    
    policy_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss=losses,loss_weights=lossWeights)
    policy_model.fit(prev_state,[np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1)])

    speed,steer,muSpeed,sigmaSpeed,muSteer,sigmaSteer = [None for x in range(6)]
    prev_state = None
    iteration += 1


  elif (os.path.isfile(f"/content/drive/MyDrive/RL4NXP/0_{iteration}_1.jpeg")):
    passedImage = cv2.imread(f"/content/drive/MyDrive/RL4NXP/0_{iteration}_1.jpeg")

    prev_V = value_model.predict(prev_state)
    current_V = value_model.predict(passedImage)

    reward = -1500
    target = reward + gamma*current_V
    td_error = target - prev_V
    value_model.fit(x= prev_state, y = target)

    losses={'speed':policy_loss(muSpeed,sigmaSpeed,td_error),'steer':policy_loss(muSteer,sigmaSteer,td_error),"muSpeed" :policy_loss(muSteer,sigmaSteer,td_error),"sigmaSpeed" :policy_loss(muSteer,sigmaSteer,td_error),"muSteer":policy_loss(muSteer,sigmaSteer,td_error),"sigmaSteer":policy_loss(muSteer,sigmaSteer,td_error)}
    lossWeights={'speed':1,'steer':1,"muSpeed":0,"sigmaSpeed":0,"muSteer":0,"sigmaSteer":0}
    
    policy_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss=losses,loss_weights=lossWeights)
    policy_model.fit(prev_state,[np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1),np.array(0.0).reshape(-1,1)])

    speed,steer,muSpeed,sigmaSpeed,muSteer,sigmaSteer = [None for x in range(6)]
    prev_state = None
    iteration += 1



KeyboardInterrupt: ignored

In [21]:
###Debugs##


# speed,steer,muSpeed,sigmaSpeed,muSteer,sigmaSteer = [None for x in range(6)]
# reward = -1+251
# file1 = drive.CreateFile({'title': f"1_25_2R.txt" , 'parents':[{'id':'1U1tIilfX2rzKiQidcmEQEgaATi-Y2trt'}]})
# file1.SetContentString(f"{reward}")
# file1.Upload()
# file1 = open(f"/content/drive/MyDrive/RL4NXP/1_25_2R.txt","r")
# reward=float(file1.readline())
# reward

# iteration = 0
# img = cv2.imread("/content/drive/MyDrive/03.jpeg")
# img= np.array(img) 
# img = cv2.resize(img,(64,64))
# img = np.reshape(img,(1,64,64,3))
# speed,steer,muSpeed,sigmaSpeed,muSteer,sigmaSteer = policy_model.predict(img)
# print(speed,steer,muSpeed,sigmaSpeed,muSteer,sigmaSteer )
# file1 = drive.CreateFile({'title': f"1_{iteration}_2.txt" , 'parents':[{'id':'1U1tIilfX2rzKiQidcmEQEgaATi-Y2trt'}]})
# file1.SetContentString(f"{float(speed)} {float(steer)} {float(muSpeed)} {float(sigmaSpeed)} {float(muSteer)} {float(sigmaSteer)}")
# file1.Upload()

# # file1 = open(f"/content/drive/MyDrive/0{iteration}2P.jpeg","w+")
# # file1.writelines(f"{float(speed)} {float(steer)} {float(muSpeed)} {float(sigmaSpeed)} {float(muSteer)} {float(sigmaSteer)}") 
# # file1.close()
# # file1 = open(f"/content/drive/MyDrive/0{iteration}2P.jpeg","r")
# # a = file1.readline()
# # b=a.split(" ")
# # c,d,f,g,h, i = [float(x) for x in b]
# # print(c,d,f,g,h, i)

250.0